<a href="https://colab.research.google.com/github/Hanbin-git/practice/blob/main/upbit1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyupbit

In [ ]:
# API 키 설정
access = ""
secret = ""
upbit = pyupbit.Upbit(access, secret)

In [ ]:
import pyupbit
import time
import datetime
import os
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import numpy as np

# API 키 설정
access = "Nw2bQfQdCj5yIVKBlMlxIqbv0zWXCSZk2YoUxsCL"
secret = "4K6ZgQWUQ8AXqphJO7x5n0sGUjr9WLWVUxqqVQTy"
upbit = pyupbit.Upbit(access, secret)


SAVE_DIR = "E:/crypto_log"
os.makedirs(SAVE_DIR, exist_ok=True)
log_path = os.path.join(SAVE_DIR, "trade_log.txt")
chart_path = os.path.join(SAVE_DIR, "asset_chart.png")

# 초기 상태
cash = 1_000_000
holding = 0
buy_price = 0
prev_price = None
last_trade_time = None
last_action = None

# 수수료 설정
buy_fee = 0.0005
sell_fee = 0.0005

# RSI 조건 (완화된 시작값)
rsi_buy_threshold = 50
rsi_sell_threshold = 60

time_log, asset_log = [], []

def get_rsi(ticker, window=14):
    df = pyupbit.get_ohlcv(ticker, interval="minute60", count=200)
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window).mean()
    avg_loss = loss.rolling(window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi.iloc[-1]

ticker = "KRW-XRP"

while True:
    try:
        now = datetime.datetime.now()
        rsi = get_rsi(ticker)
        price = pyupbit.get_current_price(ticker)
        now_str = now.strftime('%Y-%m-%d %H:%M:%S')

        # 가격 변화율
        price_change_pct = abs((price - prev_price) / prev_price * 100) if prev_price else 0
        prev_price = price

        # 변동성 계산
        if len(asset_log) >= 3:
            volatility = np.std(asset_log[-3:]) / price * 100
        else:
            volatility = 0

        # 매매 간격 조건
        time_ok = (last_trade_time is None) or ((now - last_trade_time).seconds > 60 * 30)

        msg = f"[{now_str}] [관망] RSI: {rsi:.2f}, 가격: {price:.1f}, 총 자산: {cash + holding * price:,.0f}원"

        # 매수 조건
        if rsi < rsi_buy_threshold and cash > 0 and time_ok:
            amount = cash * (1 - buy_fee)
            qty = amount / price
            holding += qty
            buy_price = price if buy_price == 0 else (buy_price + price) / 2
            cash = 0
            last_trade_time = now
            last_action = "buy"
            msg = f"[{now_str}] ✅ [매수] RSI: {rsi:.2f}, 가격: {price}, 수량: {qty:.4f}"
            # 조건 변화: 매도 조건을 더 까다롭게
            rsi_sell_threshold = min(rsi_sell_threshold + 1, 80)

        # 매도 조건
        elif rsi > rsi_sell_threshold and holding > 0 and time_ok:
            qty = holding
            amount = qty * price * (1 - sell_fee)
            profit = amount - (buy_price * qty)
            cash = amount
            holding = 0
            last_trade_time = now
            last_action = "sell"
            msg = f"[{now_str}] ✅ [매도] RSI: {rsi:.2f}, 가격: {price}, 수익: {profit:,.0f}원"

            # 조건 변화: 손해면 매수는 더 조심하게, 매도는 더 빨리
            if profit < 0:
                rsi_buy_threshold = min(rsi_buy_threshold + 1, 60)
                rsi_sell_threshold = max(rsi_sell_threshold - 1, 50)

        print(msg)
        with open(log_path, "a", encoding="utf-8") as f:
            f.write(msg + "\n")

        # 자산 그래프
        total_asset = cash + holding * price
        time_log.append(now_str)
        asset_log.append(total_asset)

        if len(asset_log) > 1:
            plt.figure(figsize=(10, 4))
            plt.plot(time_log, asset_log, marker='o')
            plt.title("자산 변화 추이")
            plt.xlabel("시간")
            plt.ylabel("KRW")
            plt.xticks(rotation=45)
            plt.grid(True)
            plt.tight_layout()
            plt.savefig(chart_path)
            plt.close()

    except Exception as e:
        print(f"[에러] {e}")

    time.sleep(10)
